In [1]:
from libs.GPT import GPT
from libs.CorpusDataset import CorpusDataset
import torch
import torch.nn as nn
from transformers import AutoTokenizer
import glob
torch.set_default_device('cuda')

In [2]:
tokenizer = AutoTokenizer.from_pretrained("ikit-claw-nlp/toy-llm")
GPT_CONFIG_124M = {
    "vocab_size": tokenizer.vocab_size,
    "pad_idx": tokenizer.convert_tokens_to_ids("<pad>"),
    "context_length": 1024, #max context length
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}


In [5]:
model = GPT(GPT_CONFIG_124M)

In [85]:
user_inputs = 'こんにちは、わたしは'

In [86]:
user_inputs_ids = tokenizer.encode(user_inputs, return_tensors="pt")

In [ ]:
max_new_token_num = 10
def generate_new_tokens(model, user_input_ids, context_size, max_new_token_num):
    for _ in range(0, max_new_token_num):
        # user_input_ids = [batch_size, sequence_length]
        token_ids = user_input_ids[:, -context_size:]
        # logits = [batch_size, max_context_length, dimension_size]
        with torch.no_grad():
            logits = model(token_ids)
        # logits = [batch_size, 1, dimension_size] => we only care about the last token.
        logits = logits[:, -1, :]
        # genera
        next_token_ids = torch.argmax(logits, dim=-1, keepdim=True)
        user_input_ids = torch.cat((user_input_ids, next_token_ids), dim=1)
    return user_input_ids

In [98]:
generate_new_tokens(model=model, user_input_ids=user_inputs_ids, context_size=1024, max_new_token_num=10)

In [90]:
tokenizer.decode(user_inputs_ids[0, :])

'こんにちは、わたしは'

In [26]:
tokenizer.convert_ids_to_tokens(user_inputs_ids)

ValueError: invalid literal for int() with base 10: 'input_ids'